In [17]:
import pandas as pd

## CMA Population Changes from 2011 to 2016

In [ ]:
raw = pd.read_csv("raw/2016_census.csv")

In [ ]:
population_raw = raw[raw["DIM: Profile of Census Metropolitan Areas/Census Agglomerations (2247)"].isin(["Population, 2016", "Population, 2011"])]

display(population_raw)

In [ ]:
population_pivot = pd.pivot(population_raw, index=["GEO_CODE (POR)", "GEO_NAME"], columns="DIM: Profile of Census Metropolitan Areas/Census Agglomerations (2247)", values="Dim: Sex (3): Member ID: [1]: Total - Sex")
population_pivot["pct_change"] = (population_pivot["Population, 2016"].astype(int) - population_pivot["Population, 2011"].astype(int)) / population_pivot["Population, 2011"].astype(int)
display(population_pivot)

population_pivot.to_csv("exports/populations2016.csv")

In [ ]:
cmas = population_raw[["ALT_GEO_CODE", "GEO_NAME"]].drop_duplicates(subset="ALT_GEO_CODE")
code = cmas["ALT_GEO_CODE"].astype(str).str[-3:].astype(int)
cmas["ALT_GEO_CODE"] = code


display(cmas)
cmas.to_csv("exports/cmas_list.csv")

## Census subdivisions - Change from 2011 to 2016

In [ ]:
raw_csd = pd.read_csv("raw/2016_census_csd.csv")

display(raw_csd)

In [ ]:
csd = raw_csd[raw_csd["GEO_LEVEL"] == 3]
csd = csd[csd["DIM: Profile of Census Divisions/Census Subdivisions (2247)"] == "Population percentage change, 2011 to 2016"]

display(csd)

In [ ]:
csd_pivot = pd.pivot(csd, index=["GEO_CODE (POR)", "GEO_NAME"], columns="DIM: Profile of Census Divisions/Census Subdivisions (2247)", values="Dim: Sex (3): Member ID: [1]: Total - Sex").reset_index()
display(csd_pivot)
csd_pivot.to_csv("exports/census2016.csv")

## Census tracts - Change from 2011 to 2016

In [ ]:
tracts_raw = pd.read_csv("raw/2016_census_tracts.csv")
tracts_population = tracts_raw[tracts_raw["DIM: Profile of Census Tracts (2247)"] == "Population percentage change, 2011 to 2016"]
tracts_population = tracts_population[tracts_population["GEO_LEVEL"] == 2]

display(tracts_population)

In [ ]:
tracts_pivot = pd.pivot(tracts_population, columns="DIM: Profile of Census Tracts (2247)", index=["GEO_CODE (POR)", "GEO_NAME"], values="Dim: Sex (3): Member ID: [1]: Total - Sex").reset_index().set_index("GEO_CODE (POR)")
tracts_pivot["CMA_code"] = tracts_pivot.index.astype(str).str[0:3].astype(int)
tracts_pivot = tracts_pivot.merge(cmas, how="left", left_on="CMA_code", right_on="ALT_GEO_CODE")
tracts_pivot = tracts_pivot[["GEO_NAME_x", "GEO_NAME_y", "Population percentage change, 2011 to 2016"]]

display(tracts_pivot)
tracts_pivot.to_csv('exports/censustracts_change.csv')